## Softmax方策を貪欲方策で近似する

RLではたまに確率的な方策しか使えないことがありますが，Softmax方策はパラメータによっては貪欲方策として扱えます．

参考：
* [Towards Achieving Sub-linear Regret and Hard Constraint Violation in Model-free RL](https://proceedings.mlr.press/v238/ghosh24a.html)

準備：
* $Q: \mathcal{A}\to \mathbb{R}$
* Softmax：$\pi(\cdot) \propto \exp(\alpha Q(\cdot))$
* $a' = \arg \max_a Q(a)$
* $V_{\max} = \max_a Q(a)$
* $V_{\pi} = \sum_a \pi(a) Q(a)$

まず，Log-sum-expの不等式から，
$$
0 \leq \frac{1}{\alpha}\log \sum_a \exp(\alpha Q_a) - Q(a') 
$$
この不等式はWikiにあります．


これを使うと，
$$
\begin{aligned}
V_{\max} - V_\pi = Q(a') - \sum_a \pi(a) Q(a)
&\leq \frac{1}{\alpha}\log \sum_a \exp(\alpha Q_a) - \sum_a \pi(a) Q(a)
\end{aligned}
$$

です．ここで，https://arxiv.org/abs/1903.05926 のProposition 1を使うと，

$$
\begin{aligned}
& \frac{1}{\alpha} \sum_{a}-\pi(a) \log \left(\pi(a)\right) \\
= & \frac{1}{\alpha} \sum_a\left(-\frac{\exp({\alpha Q(a)})}{\sum_b \exp({\alpha Q(b)})} \log \left(\frac{\exp({\alpha Q(a)})}{\sum_b \exp({\alpha Q(b)})}\right)\right) \\
= & \frac{1}{\alpha} \sum_a\left(-\frac{\exp({\alpha Q(a)})}{\sum_b \exp({\alpha Q(b)})}\left(\alpha Q(a)-\log \left(\sum_b \exp({\alpha Q(b)})\right)\right)\right) \\
= & -\sum_a \frac{\exp({\alpha Q(a)}) Q(a)}{\sum_b \exp({\alpha Q(b)})}+\frac{1}{\alpha} \log \left(\sum_b \exp({\alpha Q(b)})\right) \frac{\sum_a \exp({\alpha Q(a)})}{\sum_b \exp({\alpha Q(b)})} \\
= & \frac{1}{\alpha} \log \left(\sum_b \exp({\alpha Q(b)})\right) 
-\sum_a \pi(a) Q(a)\\
\end{aligned}
$$

なので結局，
$$
V_{\max} - V_\pi
\leq \frac{1}{\alpha} \mathrm{Entropy}(\pi)
\leq \frac{\log |\mathcal{A}|}{\alpha}
$$
が成立します．

## log-sumの引き算

* [ON LOG-SUM INEQUALITIES](https://arxiv.org/pdf/2005.03272)のTheorem 1

* $a_1, \dots, a_n$と$b_1, \dots, b_n$は非負の数列
* $g$を$a_1, \dots, a_n$と$b_1, \dots, b_n$をDomainに取る実数値関数とする．
  * $g(b_0) > 0$ for all $i = 1, \dots, n$
* 関数$f: [m_g, M_g] \to \mathbb{R}$：ここで，$h(x)=xf(x)$は凸関数とする
    * $m_g:=\min _i\left\{\frac{g\left(a_i\right)}{g\left(b_i\right)}\right\}$ and $M_g:=\max _i\left\{\frac{g\left(a_i\right)}{g\left(b_i\right)}\right\}$

このとき，次が成立する：
$$\sum_{i=1}^n g\left(a_i\right) f\left(\frac{g\left(a_i\right)}{g\left(b_i\right)}\right) \geq\left(\sum_{i=1}^n g\left(a_i\right)\right) f\left(\frac{\sum_{i=1}^n g\left(a_i\right)}{\sum_{i=1}^n g\left(b_i\right)}\right)$$

**証明**

$a=\sum_{i=1}^n g\left(a_i\right)$ and $b=\sum_{i=1}^n g\left(b_i\right)>0$とする．このとき，$\frac{a}{b}=\sum_{i=1}^n \frac{g\left(b_i\right)}{b} \frac{g\left(a_i\right)}{g\left(b_i\right)}$は$\frac{g(a_i)}{g(b_i)}$の凸な組み合わせ．よって，明らかに$a/b$は$[m_g, M_g]$の中に入っている．

ここで，$g(b_i) > 0$かつ$b=\sum^n_{i=1} g(b_i)$より，$0 \leq \frac{g\left(b_i\right)}{b} \leq 1 \text { and } \sum_{i=1}^n \frac{g\left(b_i\right)}{b}=1$．
Jensenの不等式から，

$$
\sum_{i=1}^n g\left(a_i\right) f\left(\frac{g\left(a_i\right)}{g\left(b_i\right)}\right)=\sum_{i=1}^n g\left(b_i\right) \frac{g\left(a_i\right)}{g\left(b_i\right)} f\left(\frac{g\left(a_i\right)}{g\left(b_i\right)}\right)=\sum_{i=1}^n b \frac{g\left(b_i\right)}{b} h\left(\frac{g\left(a_i\right)}{g\left(b_i\right)}\right)
$$

$h$を展開すると，
$$
b h\left(\frac{a}{b}\right)=b \frac{a}{b} f\left(\frac{a}{b}\right)=a f\left(\frac{a}{b}\right)=\left(\sum_{i=1}^n g\left(a_i\right)\right) f\left(\frac{\sum_{i=1}^n g\left(a_i\right)}{\sum_{i=1}^n g\left(b_i\right)}\right)
$$
が得られる．

---

$g(x)=x$，$f(x)=\log(x)$とすると，$h(x)=x\log(x)$なので，次の不等式が得られます：

$$
\frac{\sum_{i=1}^n a_i\left\{\log \left(a_i\right)-\log \left(b_i\right)\right\} }{\sum_{j=1}^n a_j}\geq\left\{\log \left(\sum_{i=1}^n a_i\right)-\log \left(\sum_{i=1}^n b_i\right)\right\}
$$

**便利な応用**

制約付き線形MDPでCoveringを考えるときに役に立つので，次のKLとエントロピー正則化された２つの貪欲方策を考えましょう．

$$
\begin{aligned}
\pi^\prime &= \arg\max_\pi \sum_a \pi(a) Q(a) - \eta \operatorname{KL}(\pi \| \mu) + \tau \operatorname{Ent}(\pi)\\
\widehat{\pi}^\prime &= \arg\max_\pi \sum_a \pi(a) \widehat{Q}(a) - \eta \operatorname{KL}(\pi \| \widehat{\mu}) + \tau \operatorname{Ent}(\pi)\\
\end{aligned}
$$

このとき，

$$
\begin{aligned}
&\sum_a \pi^\prime(a) Q(a) - \eta \operatorname{KL}(\pi^\prime \| \mu) + \tau \operatorname{Ent}(\pi^\prime)\\
=& (\eta + \tau) \ln \sum_a \mu(a) \exp \left(\frac{1}{\eta + \tau} Q(a)\right)\\
=& (\eta + \tau) \ln \sum_a \exp \left(\underbrace{\frac{Q(a) + \eta\ln \mu(a)}{\eta + \tau}}_{=: \; q(a) / (\eta + \tau)} \right)\\
=& (\eta + \tau) \ln \sum_a \exp \left(\frac{q(a)}{\eta + \tau} \right)\\
\end{aligned}
$$

よって，
$$
\begin{aligned}
&\max_\pi \sum_a \pi(a) Q(a) - \eta \operatorname{KL}(\pi \| \mu) + \tau \operatorname{Ent}(\pi)
- \max_\pi \left(\sum_a \pi(a) \widehat{Q}(a) - \eta \operatorname{KL}(\pi \| \widehat{\mu}) + \tau \operatorname{Ent}(\pi)\right)\\
&
=\sum_a \pi^\prime(a) Q(a) - \eta \operatorname{KL}(\pi^\prime \| \mu) + \tau \operatorname{Ent}(\pi^\prime)
- \left(\sum_a \widehat{\pi}^\prime(a) \widehat{Q}(a) - \eta \operatorname{KL}(\widehat{\pi}^\prime \| \widehat{\mu}) + \tau \operatorname{Ent}(\widehat{\pi}^\prime)\right)\\
&
=(\eta + \tau) \left(\ln \sum_a \exp \left(\frac{q(a)}{\eta + \tau} \right)
-\ln \sum_a \exp \left(\frac{\widehat{q}(a)}{\eta + \tau} \right)\right)
\end{aligned}
$$

また，
$$
\begin{aligned}
\pi' \propto \exp\left(q / (\eta + \tau)\right)
\quad
\pi' \propto \exp\left(q / (\eta + \tau)\right)
\end{aligned}
$$
であることに注意しましょう．よって，

上でやったlogsumの不等式に$a_i=\exp\left(q(a) / (\eta + \tau)\right)$，$b_i = \exp \left(\widehat{q}(a) / (\eta + \tau)\right)$を代入すると，

$$
\begin{aligned}
&\leq (\eta + \tau)
\sum_a \pi^\prime(a)\left(\frac{q(a)}{\eta + \tau} - \frac{\widehat{q}(a)}{\eta + \tau}\right)\\
&\leq \max_a \left(q(a) - \widehat{q}(a)\right)\\
&= \max_a \left(Q(a) + \eta \ln \mu(a) - \widehat{Q}(a) - \eta \ln \widehat{\mu}(a)\right)
\end{aligned}
$$

が得られる．逆向きの不等式と合わせると，
$$
\begin{aligned}
&\left|\max_\pi \sum_a \pi(a) Q(a) - \eta \operatorname{KL}(\pi \| \mu) + \tau \operatorname{Ent}(\pi)
- \max_\pi \left(\sum_a \pi(a) \widehat{Q}(a) - \eta \operatorname{KL}(\pi \| \widehat{\mu}) + \tau \operatorname{Ent}(\pi)\right)\right|\\
&\leq \max_a \left|Q(a) + \eta \ln \mu(a) - \widehat{Q}(a) - \eta \ln \widehat{\mu}(a)\right|
\end{aligned}
$$


ここで，$\mu(a)$が決定的だと，$\ln \mu(a)$が無限に飛んでいってしまいます．$\eta$を小さくすれば良さそうですが，「$\eta$を小さくする＝学習率を大きくする」ことに対応するので，好ましくありません．

---

そこで，（$Q$，$\mu$），に仮定を置きましょう．
$Q^\star$と$\pi^\star$を，次を満たす（何らかのアルゴリズムの収束先の）価値と方策とします．つまり，

$$
\begin{aligned}
&\max_\pi \sum_a \pi(a\mid s) Q^\star(s, a) - \eta \operatorname{KL}(\pi \| \pi^\star)_s + \tau \operatorname{Ent}(\pi)_s\\
= 
&\sum_a \pi^\star(a\mid s) \left(Q^\star(s, a) - \tau \ln \pi^\star(a \mid s)\right)\quad  \forall s \in \mathcal{S}\;.
\end{aligned}
$$

が満たされています．そして，$\pi$と$Q$は上式の値から$\varepsilon$だけずれた組とすれば，多分バウンドできるはずです．
そして，$\pi^\star$は

$$
\pi^\star \propto \exp\left(\frac{Q^\star + \eta \ln \pi^\star}{\eta + \tau}\right)
\propto \left(\pi^\star\right)^{\frac{\eta}{\eta + \tau}}
\exp\left(\frac{Q^\star}{\eta + \tau}\right)
$$

を満たすので，結局
$$
\pi^\star \propto 
\exp\left(\frac{Q^\star}{\tau}\right)
$$
になるので，$Q^\star$のsoftmax方策になります．

あとは**何らかのアルゴリズムが全ての$s$で収束する**ことを示し，**アルゴリズムのV値が最適な値と$\varepsilon$だけずれる**ことを示せば良さそうです．（多分全ての$s$で示すのは無理〜　そもそもOptimismの証明を変えないといけないかも）
